In [2]:
import pandas as pd 
from sklearn import preprocessing
import numpy as np
import ast
import networkx as nx
from collections import Counter
import itertools


In [6]:
from utilities import *

In [3]:
# Aggregation Rule : Plurality
# Tie breaker 1: Pick the solution that gives maximum utility
# Tie breaker 2: Pick the one that gives most increase from previous solution

# Q: How realistic is the fact that their only agency is a vote?

In [7]:
payoffs = load_data('../output/processed/txt/bestruns100years_newcoalitions.txt', 'old')
payoffs.columns

Index(['hydropower_revenue', 'atomic_power_plant_discharge',
       'baltimore_discharge', 'chester_discharge', 'recreation', 'environment',
       'flood_duration'],
      dtype='object')

In [16]:
normalized_payoffs = normalize_df(payoffs)

In [20]:
normalized_payoffs = normalized_payoffs.set_index('decision')

In [23]:
sum(normalized_payoffs.T['best_dfl'])

1.3494244809064195

In [25]:
normalized_payoffs.T['best_dfl']

hydropower_revenue              0.349424
atomic_power_plant_discharge    0.000000
baltimore_discharge             0.000000
chester_discharge               0.000000
recreation                      0.000000
environment                     1.000000
flood_duration                  0.000000
Name: best_dfl, dtype: float64

In [26]:
normalized_payoffs.T['best_comp']

hydropower_revenue              1.000000
atomic_power_plant_discharge    0.998935
baltimore_discharge             0.696771
chester_discharge               0.961838
recreation                      0.920400
environment                     0.787858
flood_duration                  0.990099
Name: best_comp, dtype: float64

In [24]:
sum(normalized_payoffs.T['best_comp'])

6.355900971782913

In [98]:
def mode(vote):
    count = Counter(vote)
    max_value = max(count.values())
    return [k for k,v in count.items() if v == max_value]


In [ ]:
def calculate_winning_decision(vote):
    max_vote = mode(vote)
    if len(max_vote) == 1:
        return max_vote[0]
    else:
        utilities = {k:v['total_utility'] for k,v in npayoffs.items() if k in max_vote}
        max_utility = [k for k,v in utilities.items() if v == max(utilities.values())]
        if len(max_utility) == 1:
            return max_utility[0]
        else:
            return max_utility[0]
            print("same max utilities")

In [100]:
def calculate_dominant_strategies(player_list, payoffs):
    payoffs = payoffs[player_list]
    normalized_payoffs = utils.normalize_df(payoffs)

    normalized_payoffs['total_utility'] = normalized_payoffs[player_list].sum(axis = 1)

    decisions = list(normalized_payoffs['decision'])

    npayoffs = normalized_payoffs.T.to_dict()

    x = list(range(len(decisions)))
    n_players = len(player_list)
    all_combinations = [p for p in itertools.product(x, repeat = n_players)]

    utility = {}
    for i in range(len(all_combinations)):
        utility[i] = {'vote': all_combinations[i], 'decision': npayoffs[calculate_winning_decision(all_combinations[i])]['decision'] }

    strategies = {}
    for i in range(n_players):
        strategies[i] = {}
        for j in range(len(decisions)):
            strategies[i][j] = set([utility[k]['decision'] for k,v in utility.items() if utility[k]['vote'][i] == j])

    decision_overview = {}
    for i in decisions:
        decision_overview[i] = len([utility[k]['vote'] for k,v in utility.items() if utility[k]['decision'] == i])

    return strategies, utility, decision_overview

In [106]:
x = list(range(len(decisions)))
n_players = len(actors)
all_combinations = [p for p in itertools.product(x, repeat = n_players)]

In [108]:
len(all_combinations)

823543

In [104]:
payoffs.columns

Index(['coalition_appchester', 'coalition_discharge', 'coalition_watersupply',
       'hydropower_revenue', 'atomic_power_plant_discharge',
       'baltimore_discharge', 'chester_discharge', 'recreation', 'environment',
       'flood_risk', 'flood_duration'],
      dtype='object')

In [105]:
decisions = ['best_app',
 'best_ch',
 'best_comp',
 'best_dfl',
 'best_env',
 'best_hydro',
 'best_rec']
actors = ['hydropower_revenue', 'atomic_power_plant_discharge',
       'baltimore_discharge', 'chester_discharge', 'recreation', 'environment','flood_duration']

In [ ]:
vote_space_1 = np.array()

In [18]:
def game(payoffs, player1, player2):
    twoplayers = payoffs[[player1, player2]]
    normalized_payoffs = utils.normalize_df(twoplayers)
    normalized_payoffs = normalized_payoffs.set_index('index')
    normalized_payoffs.reset_index(drop=True)

    from non_cooperative_games import noncooperativegames
    noncooperativegames = noncooperativegames(player1, player2)
    vote_space_p1, vote_space_p2, vote_outcomes = noncooperativegames.votes(normalized_payoffs)
    G = noncooperativegames.graphs(vote_space_p1, vote_space_p2)

    return G, vote_outcomes, normalized_payoffs

In [19]:
from non_cooperative_games import noncooperativegames
player2 = 'chester_discharge'
player1 = 'flood_duration'
noncooperativegames = noncooperativegames(player1, player2)
G, vote_outcomes, normalized_payoffs = game(payoffs, player1, player2)

equilibrium = list(dict.fromkeys([vote_outcomes[n]['decision'] for n,v in G.nodes(data = True) if v['out_arrows'] == 0]))
print(f'equilibrium solutions according to plurality are {equilibrium}')
gmr_p1 = noncooperativegames.gmr(G, 1,'u1')
print(f'GMR for {player1} is', [vote_outcomes[x] for x in gmr_p1])
gmr_p2 = noncooperativegames.gmr(G, 2,'u2')
print(f'GMR for {player2} is', [vote_outcomes[x] for x in gmr_p2])

equilibrium solutions according to plurality are ['best_rec']
GMR for flood_duration is []
GMR for chester_discharge is []


In [20]:
normalized_payoffs

,decision,flood_duration,chester_discharge
index,,,
0,best_app,0.990099,0.996224
1,best_ch,0.990099,0.956463
2,best_comp,0.990099,0.961838
3,best_dfl,0.000000,0.000000
4,best_env,0.990099,0.998633
5,best_hydro,0.990099,0.988003
6,best_rec,1.000000,1.000000


In [422]:
# equilibria_p2 = []
# for n in G.nodes:
#     intermediate_nodes = [x[1] for x in list(nx.dfs_edges(G, source = n, depth_limit = 1)) if G.edges[x]['player'] == 2]
#     for i in intermediate_nodes:
#         edges_list = list(nx.dfs_edges(G, source = i, depth_limit = 1))
#         tertiary_nodes = [edge[1] for edge in edges_list if G.edges[edge]['player'] == 1 and G.edges[edge]['edge_type'] == 'useful']
#         for j in tertiary_nodes:
#             #print(j)
#             last_nodes = [G.nodes[x[1]]['u2'] for x in list(nx.dfs_edges(G, source = j, depth_limit = 1)) if G.edges[x]['player'] == 2]
#             #print(last_nodes)
#             if all(last_nodes < G.nodes[n]['u2']):
#                 equilibria_p2.append(n)
# equilibria_p2 = list(dict.fromkeys(equilibria_p2))

In [53]:
for n in G.nodes:
    intermediate_nodes = [x[1] for x in list(nx.dfs_edges(G, source = n, depth_limit = 1)) if G.edges[x]['player'] == 1 and G.edges[x]['edge_type'] == 'useful']
    print(intermediate_nodes)

[7, 14, 21, 28, 42]
[]
[]
[]
[11, 18, 46]
[12, 19, 26, 33, 40, 47]
[13, 20]
[14, 21]
[]
[2, 16, 23, 30, 37, 44]
[3, 17, 24, 38]
[18]
[19, 26, 40]
[20]
[21]
[]
[]
[3, 24, 38]
[]
[26]
[]
[]
[]
[]
[]
[11, 18, 46]
[]
[13, 20]
[7, 14, 21, 42]
[]
[]
[3, 10, 17, 24, 38, 45]
[11, 18, 46]
[12, 19, 26, 40, 47]
[13, 20]
[7, 14, 21, 28, 42]
[]
[]
[]
[11, 18, 46]
[19, 26]
[13, 20]
[7, 14, 21]
[]
[]
[3, 10, 17, 24, 38]
[11, 18]
[12, 19, 26, 40]
[13, 20]


In [72]:
intermediate_nodes = [x[1] for x in list(nx.dfs_edges(G, source = 42, depth_limit = 1)) if G.edges[x]['player'] == 1 and G.edges[x]['edge_type'] == 'useful']
print(intermediate_nodes)
tertiary_nodes = [edge[1] for edge in list(nx.dfs_edges(G, source = 0, depth_limit = 1)) if G.edges[edge]['player'] == 2 and G.edges[edge]['edge_type'] == 'useful']
print(tertiary_nodes)
last_nodes = [G.nodes[x]['u1'] for x in tertiary_nodes]

print(G.nodes[42]['u1'])
# #last_nodes = [G.nodes[x[1]]['u2'] for x in list(nx.dfs_edges(G, source = 44, depth_limit = 1)) if G.edges[x]['player'] == 2]
#print(last_nodes)
# if all(last_nodes < G.nodes[22]['u1']):
#     print(True)

[0, 7, 14, 21, 28, 35]
[6]
0.9987639934185715
